imports & connection

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import sqlite3

DWengine = create_engine("mssql+pyodbc://LEGION\\SQLEXPRESS/Data Warehouse?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

Create dataframes from DB tables

In [23]:
course_df = pd.read_sql_query('SELECT * FROM course', DWengine)
order_df = pd.read_sql_query('SELECT * FROM [order]', DWengine)
order_details_df = pd.read_sql_query('SELECT * FROM order_details', DWengine)
order_method_df = pd.read_sql_query('SELECT * FROM order_method', DWengine)
retailer_site_df = pd.read_sql_query('SELECT * FROM retailer_site', DWengine)
return_reason_df = pd.read_sql_query('SELECT * FROM return_reason', DWengine)
returned_item_df = pd.read_sql_query('SELECT * FROM returned_item', DWengine)
sales_staff_df = pd.read_sql_query('SELECT * FROM sales_Staff', DWengine)
training_df = pd.read_sql_query('SELECT * FROM training', DWengine)

Source Data Model connection

In [ ]:
SDMengine = create_engine("mssql+pyodbc://LEGION\\SQLEXPRESS/sdm?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

course table data insert

In [ ]:
course_sdm_df = pd.read_sql_query('SELECT * FROM course', SDMengine)

course_sdm_df.to_sql('course', con=DWengine, if_exists='append', index=False) 

return_reason table data insert

In [ ]:
return_reason_sdm_df = pd.read_sql_query('SELECT * FROM return_reason', SDMengine)

return_reason_sdm_df.to_sql('return_reason', con=DWengine, if_exists='append', index=False)

order_method table data insert

In [ ]:
order_method_sdm_df = pd.read_sql_query('SELECT * FROM order_method', SDMengine)

order_method_sdm_df.to_sql('order_method', con=DWengine, if_exists='append', index=False)

retailer_site table data insert

In [27]:
retailer_site_sdm_df = pd.read_sql_query('SELECT * from retailer_site', SDMengine)
country_sdm_df = pd.read_sql_query('SELECT * FROM country', SDMengine)

merged_retailer_site_df = retailer_site_sdm_df.merge(country_sdm_df, on=['COUNTRY_CODE','COUNTRY_CODE'])


,RETAILER_SITE_CODE,RETAILER_CODE,ADDRESS1,ADDRESS2,CITY,REGION,COUNTRY_CODE,ACTIVE_INDICATOR,COUNTRY_EN,LANGUAGE,CURRENCY_NAME,FLAG_IMAGE,SALES_TERRITORY_CODE
0,1,89,1117 Franklin Blvd,None,Winnipeg,Manitoba,4,1,Canada,EN,dollars,F04,1
1,2,89,"45, rue Atwater",None,Montréal,Québec,4,1,Canada,EN,dollars,F04,1
2,3,89,328 Hodgson Road,None,Fredericton,New Brunswick,4,1,Canada,EN,dollars,F04,1
3,4,90,"2425, rue Peel",None,Montréal,Québec,4,1,Canada,EN,dollars,F04,1
4,5,90,34 8th Avenue,None,Sudbury,Ontario,4,1,Canada,EN,dollars,F04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,439,209,Hüttenstraße 87,None,Klosterneuburg,None,5,1,Austria,EN,schillings,F05,7
387,440,209,Hauptstraße 85,None,Linz,None,5,1,Austria,EN,schillings,F05,7
388,441,209,Amselweg 8,None,Innsbruck,None,5,1,Austria,EN,schillings,F05,7
389,442,209,Hasengasse 38,None,Ischgl,None,5,1,Austria,EN,schillings,F05,7
